# Clustering Crypto

In [ ]:
# Initial imports
import pandas as pd
import hvplot.pandas
from path import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans


### Deliverable 1: Preprocessing the Data for PCA

In [ ]:
# Load the crypto_data.csv dataset.
file_path='Resources/crypto_data.csv'
data_df=pd.read_csv(file_path,encoding='ISO-8859-1')

data_df=data_df.set_index('Unnamed: 0')
data_df=data_df.rename_axis(None, axis=0)
data_df.head() # YOUR CODE HERE

In [221]:
# Keep all the cryptocurrencies that are being traded.
# YOUR CODE HERE
crypto_df=data_df.loc[data_df['IsTrading'] == True]

In [222]:
for column in crypto_df.columns:
    print(f'Column {column} has {crypto_df[column].isnull().sum()} null values')

Column CoinName has 0 null values
Column Algorithm has 0 null values
Column IsTrading has 0 null values
Column ProofType has 0 null values
Column TotalCoinsMined has 459 null values
Column TotalCoinSupply has 0 null values


In [223]:
# Keep all the cryptocurrencies that have a working algorithm.
# YOUR CODE HERE
crypto_df=crypto_df.loc[crypto_df['Algorithm']!='N/A']

In [224]:
# Remove the "IsTrading" column. 
# YOUR CODE HERE
crypto_df=crypto_df.drop(['IsTrading'], axis=1)

In [225]:
# Remove rows that have at least 1 null value.
# YOUR CODE HERE
crypto_df=crypto_df.dropna()
crypto_df

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
...,...,...,...,...,...
ZEPH,ZEPHYR,SHA-256,DPoS,2.000000e+09,2000000000
GAP,Gapcoin,Scrypt,PoW/PoS,1.493105e+07,250000000
BDX,Beldex,CryptoNight,PoW,9.802226e+08,1400222610
ZEN,Horizen,Equihash,PoW,7.296538e+06,21000000


In [226]:
# Keep the rows where coins are mined.
# YOUR CODE HERE
crypto_df=crypto_df.loc[crypto_df['TotalCoinsMined']!=0]
crypto_df

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0
...,...,...,...,...,...
ZEPH,ZEPHYR,SHA-256,DPoS,2.000000e+09,2000000000
GAP,Gapcoin,Scrypt,PoW/PoS,1.493105e+07,250000000
BDX,Beldex,CryptoNight,PoW,9.802226e+08,1400222610
ZEN,Horizen,Equihash,PoW,7.296538e+06,21000000


In [227]:
# Create a new DataFrame that holds only the cryptocurrencies names.
# YOUR CODE HERE
coin_df= pd.DataFrame(crypto_df['CoinName'])
coin_df.head()

,CoinName
42,42 Coin
404,404Coin
1337,EliteCoin
BTC,Bitcoin
ETH,Ethereum


In [228]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
# YOUR CODE HERE
crypto_df=crypto_df.drop(['CoinName'], axis=1)
crypto_df

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0
...,...,...,...,...
ZEPH,SHA-256,DPoS,2.000000e+09,2000000000
GAP,Scrypt,PoW/PoS,1.493105e+07,250000000
BDX,CryptoNight,PoW,9.802226e+08,1400222610
ZEN,Equihash,PoW,7.296538e+06,21000000


In [229]:
# Use get_dummies() to create variables for text features.
# YOUR CODE HERE
converted_df = pd.get_dummies(data=crypto_df, columns=['Algorithm', 'ProofType'])
X=pd.concat([crypto_df.drop(columns=['Algorithm', 'ProofType'],axis=1),converted_df],axis=1)
X.head()

,TotalCoinsMined,TotalCoinSupply,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
42,4.199995e+01,42,4.199995e+01,42,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,532000000,1.055185e+09,532000000,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,314159265359,2.927942e+10,314159265359,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,21000000,1.792718e+07,21000000,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,1.076842e+08,0,1.076842e+08,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [230]:
# Standardize the data with StandardScaler().
# YOUR CODE HERE
crypto_scaled=StandardScaler().fit_transform(X)
print(crypto_scaled[0:5])

[[-0.11674788 -0.15286468 -0.11674788 -0.15286468 -0.0433555  -0.0433555
  -0.0433555  -0.06137164 -0.07523548 -0.0433555  -0.06137164 -0.06137164
  -0.0433555  -0.0433555  -0.19226279 -0.06137164 -0.09731237 -0.0433555
  -0.11536024 -0.07523548 -0.0433555  -0.0433555  -0.15176505 -0.0433555
  -0.13105561 -0.0433555  -0.0433555  -0.08695652 -0.0433555  -0.0433555
  -0.0433555  -0.0433555  -0.06137164 -0.0433555  -0.08695652 -0.08695652
  -0.08695652 -0.0433555  -0.13105561 -0.13827675 -0.13827675 -0.0433555
  -0.06137164 -0.0433555  -0.07523548 -0.1815096  -0.0433555  -0.0433555
  -0.0433555  -0.07523548 -0.15811388 -0.3145935  -0.0433555  -0.08695652
  -0.07523548 -0.06137164 -0.0433555   1.38873015 -0.0433555  -0.0433555
  -0.06137164 -0.0433555  -0.0433555  -0.0433555  -0.0433555  -0.0433555
  -0.0433555  -0.0433555  -0.0433555  -0.39836623 -0.0433555  -0.1815096
  -0.0433555  -0.08695652 -0.08695652 -0.10670145 -0.0433555  -0.0433555
  -0.13105561 -0.0433555  -0.0433555  -0.0433555

### Deliverable 2: Reducing Data Dimensions Using PCA

In [231]:
# Using PCA to reduce dimension to three principal components.
# YOUR CODE HERE
pca = PCA(n_components=3)
crypto_pca = pca.fit_transform(crypto_scaled)
crypto_pca

array([[-0.32740681,  1.10100116, -0.50098729],
       [-0.29974784,  1.10196958, -0.50099309],
       [ 4.18903698,  1.57004613, -0.47905056],
       ...,
       [ 0.16552525, -2.30960536,  0.35269272],
       [-0.26528434, -2.10606273,  0.32959281],
       [-0.31935433,  0.82303168, -0.26149034]])

In [232]:
# Create a DataFrame with the three principal components.
# YOUR CODE HERE
pcs_df=pd.DataFrame(
    data=crypto_pca, columns=['PC 1','PC 2','PC 3'],index=X.index
)
pcs_df

,PC 1,PC 2,PC 3
42,-0.327407,1.101001,-0.500987
404,-0.299748,1.101970,-0.500993
1337,4.189037,1.570046,-0.479051
BTC,-0.261283,-1.330548,0.141688
ETH,-0.267302,-1.977670,0.290905
...,...,...,...
ZEPH,1.231182,0.667004,0.302005
GAP,-0.324021,1.100793,-0.500980
BDX,0.165525,-2.309605,0.352693
ZEN,-0.265284,-2.106063,0.329593


### Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [233]:
# Create an elbow curve to find the best value for K.
# YOUR CODE HERE
# Store Values of K to Plot
inertia=[]
k=list(range(1,11))

# Loop Through K Values and Find Inertia for best k value
for i in k:
    km=KMeans(n_clusters=i, random_state=0)
    km.fit(pcs_df)
    inertia.append(km.inertia_)
    

In [234]:
elbow_data={'k':k,'inertia':inertia}
df_elbow=pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x='k',y='inertia',title='Elbow Curve',xticks=k)

:Curve   [k]   (inertia)

Running K-Means with `k=4`

In [235]:
# Initialize the K-Means model.
# YOUR CODE HERE
model=KMeans(n_clusters=4,random_state=0)

# Fit the model
# YOUR CODE HERE
model.fit(pcs_df)

# Predict clusters
# YOUR CODE HERE
predictions=model.predict(pcs_df)
predictions

array([0, 0, 0, 3, 3, 3, 0, 3, 3, 3, 0, 3, 0, 0, 3, 0, 3, 3, 0, 0, 3, 3,
       3, 3, 3, 0, 3, 3, 3, 0, 3, 0, 3, 3, 0, 0, 3, 3, 3, 3, 3, 3, 0, 0,
       3, 3, 3, 3, 3, 0, 0, 3, 0, 3, 3, 3, 3, 0, 3, 3, 0, 3, 0, 0, 0, 3,
       3, 3, 0, 0, 0, 0, 0, 3, 3, 3, 0, 0, 3, 0, 3, 0, 0, 3, 3, 3, 3, 0,
       0, 3, 0, 3, 3, 0, 0, 3, 0, 0, 3, 3, 0, 0, 3, 0, 0, 3, 0, 3, 0, 3,
       0, 3, 0, 0, 3, 3, 0, 3, 3, 3, 0, 3, 3, 3, 3, 3, 0, 0, 3, 3, 3, 0,
       3, 0, 3, 3, 0, 3, 0, 3, 0, 0, 3, 3, 0, 3, 3, 0, 0, 3, 0, 3, 0, 0,
       0, 3, 3, 3, 3, 0, 0, 0, 0, 0, 3, 3, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0,
       0, 3, 0, 3, 0, 0, 3, 0, 3, 0, 0, 3, 0, 3, 0, 3, 0, 3, 0, 0, 0, 0,
       3, 0, 0, 0, 0, 0, 3, 3, 0, 0, 3, 3, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0,
       0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 3, 3, 3, 0, 0, 0, 0, 3, 0, 3, 0,
       0, 3, 0, 3, 3, 0, 3, 3, 0, 3, 0, 0, 0, 3, 0, 0, 3, 0, 0, 0, 0, 0,
       0, 0, 3, 0, 3, 0, 0, 0, 0, 3, 0, 3, 0, 3, 3, 3, 3, 0, 3, 0, 0, 3,
       0, 3, 3, 3, 0, 3, 0, 3, 3, 3, 0, 3, 0, 3, 0,

In [236]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
# YOUR CODE HERE
pcs_df["class"] = model.labels_
clustered_df=pd.concat([crypto_df,pcs_df,coin_df],axis=1)
#  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
# YOUR CODE HERE

#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
# YOUR CODE HERE


# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.head(10)

(533, 9)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,class,CoinName
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.327407,1.101001,-0.500987,0,42 Coin
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.299748,1.101970,-0.500993,0,404Coin
1337,X13,PoW/PoS,2.927942e+10,314159265359,4.189037,1.570046,-0.479051,0,EliteCoin
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.261283,-1.330548,0.141688,3,Bitcoin
ETH,Ethash,PoW,1.076842e+08,0,-0.267302,-1.977670,0.290905,3,Ethereum
LTC,Scrypt,PoW,6.303924e+07,84000000,-0.234167,-1.080020,-0.048832,3,Litecoin
DASH,X11,PoW/PoS,9.031294e+06,22000000,-0.386270,1.321935,-0.372110,0,Dash
XMR,CryptoNight-V7,PoW,1.720114e+07,0,-0.270410,-2.211017,0.316068,3,Monero
ETC,Ethash,PoW,1.133597e+08,210000000,-0.264595,-1.977854,0.290911,3,Ethereum Classic
ZEC,Equihash,PoW,7.383056e+06,21000000,-0.265283,-2.106063,0.329593,3,ZCash


In [266]:
clustered_df['TotalCoinSupply'] = clustered_df['TotalCoinSupply'].astype(float)
clustered_df.dtypes

Algorithm           object
ProofType           object
TotalCoinsMined    float64
TotalCoinSupply    float64
PC 1               float64
PC 2               float64
PC 3               float64
class                int32
CoinName            object
dtype: object

### Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [267]:
# Creating a 3D-Scatter with the PCA data and the clusters
# YOUR CODE HERE
fig=px.scatter_3d(clustered_df,
                  x='PC 1',
                  y='PC 2',
                  z='PC 3',
                  color='class',
                  symbol='class',
                  width=800,
)
# show plot
fig.update_layout(legend=dict(x=0,y=1))
fig.show()

In [238]:
# Create a table with tradable cryptocurrencies.
# YOUR CODE HERE
clustered_df.hvplot.table(columns=["CoinName", "Algorithm", "ProofType", "TotalCoinSupply", "TotalCoinsMined", "class"], sortable=True, selectable=True)

:Table   [CoinName,Algorithm,ProofType,TotalCoinSupply,TotalCoinsMined,class]

In [270]:
# clustered_df['TotalCoinSupply'] = clustered_df['TotalCoinSupply'].astype(float)
clustered_df.dtypes

Algorithm           object
ProofType           object
TotalCoinsMined    float64
TotalCoinSupply    float64
PC 1               float64
PC 2               float64
PC 3               float64
class                int32
CoinName            object
dtype: object

In [271]:
# Print the total number of tradable cryptocurrencies.
# YOUR CODE HERE
tradable= clustered_df['CoinName'].count()
print(f'There are {tradable} tradable cryptocurrencies')

There are 533 tradable cryptocurrencies


In [272]:
trade_df = clustered_df[['TotalCoinSupply', 'TotalCoinsMined']]
trade_df

,TotalCoinSupply,TotalCoinsMined
42,4.200000e+01,4.199995e+01
404,5.320000e+08,1.055185e+09
1337,3.141593e+11,2.927942e+10
BTC,2.100000e+07,1.792718e+07
ETH,0.000000e+00,1.076842e+08
...,...,...
ZEPH,2.000000e+09,2.000000e+09
GAP,2.500000e+08,1.493105e+07
BDX,1.400223e+09,9.802226e+08
ZEN,2.100000e+07,7.296538e+06


In [273]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.
# YOUR CODE HERE
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler().fit_transform(trade_df)
print(scaler[0:5])

[[4.20000000e-11 5.94230127e-03]
 [5.32000000e-04 7.00182308e-03]
 [3.14159265e-01 3.53420682e-02]
 [2.10000000e-05 5.96030208e-03]
 [0.00000000e+00 6.05042804e-03]]


In [276]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
# YOUR CODE HERE
plot_df=pd.DataFrame(data=scaler, columns=['TotalCoinSupply', 'TotalCoinsMined'],index=clustered_df.index)

# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
# YOUR CODE HERE
plot_df['CoinName']=clustered_df['CoinName']

# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
# YOUR CODE HERE
plot_df['class']=clustered_df['class']

plot_df.head()

,TotalCoinSupply,TotalCoinsMined,CoinName,class
42,4.200000e-11,0.005942,42 Coin,0
404,5.320000e-04,0.007002,404Coin,0
1337,3.141593e-01,0.035342,EliteCoin,0
BTC,2.100000e-05,0.005960,Bitcoin,3
ETH,0.000000e+00,0.006050,Ethereum,3


In [ ]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
# YOUR CODE HERE
plot_df.hvplot.scatter(x='TotalCoinsMined', y='TotalCoinSupply',by='CoinName',)